In [1]:
import password
import pandas as pd
import psycopg2
from psycopg2 import Error

In [2]:
POSTGRES_ADDRESS = password.getaddress()
POSTGRES_PORT = password.getport()
POSTGRES_USERNAME = password.getusername()
POSTGRES_PASSWORD = password.getpass()
POSTGRES_DBNAME = password.getdbname()


In [3]:
try:
    # Connect to an existing database
    connection = psycopg2.connect(user = POSTGRES_USERNAME,
                                  password = POSTGRES_PASSWORD,
                                  host = POSTGRES_ADDRESS,
                                  port = POSTGRES_PORT,
                                  database= POSTGRES_DBNAME)


    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
    cursor.execute("explain SELECT * from orders limit 5")
    record = cursor.fetchall()
    print("Result ", record)
    

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    

You are connected to -  ('PostgreSQL 13.2, compiled by Visual C++ build 1914, 64-bit',) 

Result  [('Limit  (cost=0.00..0.14 rows=5 width=107)',), ('  ->  Seq Scan on orders  (cost=0.00..41095.00 rows=1500000 width=107)',)]


In [4]:
def connect_sql():
    POSTGRES_ADDRESS = password.getaddress()
    POSTGRES_PORT = password.getport()
    POSTGRES_USERNAME = password.getusername()
    POSTGRES_PASSWORD = password.getpass()
    POSTGRES_DBNAME = password.getdbname()
    postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                                                                                            password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
    return postgres_str

In [18]:
qep = pd.read_sql_query(""" 
explain
select c_name , c_acctbal ,c_custkey, o_totalprice
from customer as a
join public.orders as b on a.c_custkey = b.o_custkey
where o_totalprice < 50000
""",connect_sql() )

In [19]:
qep

,QUERY PLAN
0,Hash Join (cost=7986.00..56444.84 rows=208697...
1,Hash Cond: (b.o_custkey = a.c_custkey)
2,-> Seq Scan on orders b (cost=0.00..44845....
3,Filter: (o_totalprice < '50000'::numeric)
4,-> Hash (cost=5085.00..5085.00 rows=150000...
5,-> Seq Scan on customer a (cost=0.00...


In [20]:
qep['QUERY PLAN'][4][2]

'-'

In [22]:
"\t hi"

'\t hi'

In [23]:
queries = []

In [24]:
queries = []
count = 0
for i in qep["QUERY PLAN"]:
    if "->" in i or count == 0:
        queries.append(i)
        print(i)
    count+=1

Hash Join  (cost=7986.00..56444.84 rows=208697 width=37)
  ->  Seq Scan on orders b  (cost=0.00..44845.00 rows=208697 width=12)
  ->  Hash  (cost=5085.00..5085.00 rows=150000 width=29)
        ->  Seq Scan on customer a  (cost=0.00..5085.00 rows=150000 width=29)


In [30]:
queries

['Hash Join  (cost=7986.00..56444.84 rows=208697 width=37)',
 '  ->  Seq Scan on orders b  (cost=0.00..44845.00 rows=208697 width=12)',
 '  ->  Hash  (cost=5085.00..5085.00 rows=150000 width=29)',
 '        ->  Seq Scan on customer a  (cost=0.00..5085.00 rows=150000 width=29)']

In [31]:
function = ["Hash Join", "Seq Scan", "Hash"]

In [42]:
new_dict = {}
for i in queries:
    for j in function:
        if i.find(j) != -1:
            new_dict[j] = ''
        

In [46]:
for i in queries:
    index = i.find("->")
    if index != -1:
        for j in function:
            new_index = i.find(j)
            if new_index != -1:
                print(index, j)
                break

2 Seq Scan
2 Hash
8 Seq Scan


In [36]:
for i in queries:
    print(i)
    break
    

Hash Join  (cost=7986.00..56444.84 rows=208697 width=37)


In [40]:
i.find(function[2])

0

In [8]:
cursor.close()
connection.close()
print("PostgreSQL connection is closed")

PostgreSQL connection is closed


In [138]:
example1 = """  SELECT
                                l_shipmode,
                                sum(case
                                    when o_orderpriority = '1-URGENT'
                                        OR o_orderpriority = '2-HIGH'
                                        then 1
                                    else 0
                                end) as high_line_count,
                                sum(case
                                    when o_orderpriority <> '1-URGENT'
                                        AND o_orderpriority <> '2-HIGH'
                                        then 1
                                    else 0
                                end) AS low_line_count
                            FROM
                                orders,
                                lineitem
                            WHERE
                                o_orderkey = l_orderkey
                                AND l_shipmode in ('MAIL', 'SHIP')
                                AND l_commitdate < l_receiptdate
                                AND l_shipdate < l_commitdate
                                AND l_receiptdate >= date '1994-01-01'
                                AND l_receiptdate < date '1994-01-01' + interval '1' year
                            GROUP BY
                                l_shipmode
                            ORDER BY
                                l_shipmode;"""

In [139]:
qep_new = pd.read_sql_query("explain "+ example1 ,connect_sql() )
qep_new

,QUERY PLAN
0,Finalize GroupAggregate (cost=327232.43..3274...
1,Group Key: lineitem.l_shipmode
2,-> Gather Merge (cost=327232.43..327481.45...
3,Workers Planned: 2
4,-> Partial GroupAggregate (cost=3262...
5,Group Key: lineitem.l_shipmode
6,-> Sort (cost=326232.40..32626...
7,Sort Key: lineitem.l_shipmode
8,-> Nested Loop (cost=0.4...
9,-> Parallel Seq Sca...


In [140]:
queries = []
query_explanation = []
count = 0
for i in qep_new["QUERY PLAN"]:
    if "->" in i or count == 0:
        queries.append(i)
        print(i)
    else :
        query_explanation.append(" ".join(i.split()))
    count+=1

Finalize GroupAggregate  (cost=327232.43..327481.63 rows=7 width=27)
  ->  Gather Merge  (cost=327232.43..327481.45 rows=14 width=27)
        ->  Partial GroupAggregate  (cost=326232.40..326479.81 rows=7 width=27)
              ->  Sort  (cost=326232.40..326263.32 rows=12367 width=27)
                    ->  Nested Loop  (cost=0.43..325391.81 rows=12367 width=27)
                          ->  Parallel Seq Scan on lineitem  (cost=0.00..281495.42 rows=12367 width=15)
                          ->  Index Scan using orders_pkey on orders  (cost=0.43..3.55 rows=1 width=20)


In [141]:
graphing = []

for i in queries:
    index = i.find("->")
    query_string = i.split("(")
    filter_string = " ".join(query_string[0].split())
    try:
        filter_string = filter_string.split("-> ")[1]
    except:
        filter_string = filter_string
        
    graphing.append([index, filter_string])


In [142]:
graphing

[[-1, 'Finalize GroupAggregate'],
 [2, 'Gather Merge'],
 [8, 'Partial GroupAggregate'],
 [14, 'Sort'],
 [20, 'Nested Loop'],
 [26, 'Parallel Seq Scan on lineitem'],
 [26, 'Index Scan using orders_pkey on orders']]

In [143]:
query_explanation

['Group Key: lineitem.l_shipmode',
 'Workers Planned: 2',
 'Group Key: lineitem.l_shipmode',
 'Sort Key: lineitem.l_shipmode',
 "Filter: ((l_shipmode = ANY ('{MAIL,SHIP}'::bpchar[])) AND (l_commitdate < l_receiptdate) AND (l_shipdate < l_commitdate) AND (l_receiptdate >= '1994-01-01'::date) AND (l_receiptdate < '1995-01-01 00:00:00'::timestamp without time zone))",
 'Index Cond: (o_orderkey = lineitem.l_orderkey)']

In [160]:
queries = []
index_tracker = []
count = 0
temp_query = 0

for i in qep_new["QUERY PLAN"]:
    if "->" in i or count == 0:
        queries.append(temp_query)
        
        index = i.find("->")
        if index not in index_tracker:
            index_tracker.append(index)
        query_string = i.split("(")
        filter_string = " ".join(query_string[0].split())
        
        try:
            filter_string = filter_string.split("-> ")[1]
            
        except:
            filter_string = filter_string
            
            
        temp_query = [index, filter_string]
            
    else :
        query_explanation = " ".join(i.split())
        temp_query.append(query_explanation)
        
    count+=1
    
queries.append(temp_query)

In [162]:
queries.pop(0)
for i in queries:
    i[0] = index_tracker.index(i[0])
    print(i)

[0, 'Finalize GroupAggregate', 'Group Key: lineitem.l_shipmode']
[1, 'Gather Merge', 'Workers Planned: 2']
[2, 'Partial GroupAggregate', 'Group Key: lineitem.l_shipmode']
[3, 'Sort', 'Sort Key: lineitem.l_shipmode']
[4, 'Nested Loop']
[5, 'Parallel Seq Scan on lineitem', "Filter: ((l_shipmode = ANY ('{MAIL,SHIP}'::bpchar[])) AND (l_commitdate < l_receiptdate) AND (l_shipdate < l_commitdate) AND (l_receiptdate >= '1994-01-01'::date) AND (l_receiptdate < '1995-01-01 00:00:00'::timestamp without time zone))"]
[5, 'Index Scan using orders_pkey on orders', 'Index Cond: (o_orderkey = lineitem.l_orderkey)']


In [165]:
def get_query(q_string):
    qep_new = pd.read_sql_query("explain "+ q_string ,connect_sql() )
    qep_new
    
    queries = []
    index_tracker = []
    count = 0
    temp_query = 0

    for i in qep_new["QUERY PLAN"]:
        if "->" in i or count == 0:
            queries.append(temp_query)

            index = i.find("->")
            if index not in index_tracker:
                index_tracker.append(index)
            query_string = i.split("(")
            filter_string = " ".join(query_string[0].split())

            try:
                filter_string = filter_string.split("-> ")[1]

            except:
                filter_string = filter_string


            temp_query = [index, filter_string]

        else :
            query_explanation = " ".join(i.split())
            temp_query.append(query_explanation)

        count+=1

    queries.append(temp_query)
    queries.pop(0)
    count = 0
    for i in queries:
        queries[count][0] = index_tracker.index(i[0])
        count +=1
    return queries


In [171]:
example2 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    o_orderdate,
    o_shippriority
FROM
    customer,
    orders,
    lineitem
WHERE
    c_mktsegment = 'BUILDING'
    AND c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND o_orderdate < date '1995-03-15'
    AND l_shipdate > date '1995-03-15'
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue desc,
    o_orderdate
LIMIT 20;
"""

In [172]:
get_query(example2)

[[0, 'Limit'],
 [1,
  'Sort',
  "Sort Key: (sum((lineitem.l_extendedprice * ('1'::numeric - lineitem.l_discount)))) DESC, orders.o_orderdate"],
 [2,
  'Finalize GroupAggregate',
  'Group Key: lineitem.l_orderkey, orders.o_orderdate, orders.o_shippriority'],
 [3, 'Gather Merge', 'Workers Planned: 2'],
 [4,
  'Partial GroupAggregate',
  'Group Key: lineitem.l_orderkey, orders.o_orderdate, orders.o_shippriority'],
 [5,
  'Sort',
  'Sort Key: lineitem.l_orderkey, orders.o_orderdate, orders.o_shippriority'],
 [6, 'Nested Loop'],
 [7,
  'Parallel Hash Join',
  'Hash Cond: (orders.o_custkey = customer.c_custkey)'],
 [8,
  'Parallel Seq Scan on orders',
  "Filter: (o_orderdate < '1995-03-15'::date)"],
 [8, 'Parallel Hash'],
 [9,
  'Parallel Seq Scan on customer',
  "Filter: (c_mktsegment = 'BUILDING'::bpchar)"],
 [7,
  'Index Scan using lineitem_pkey on lineitem',
  'Index Cond: (l_orderkey = orders.o_orderkey)',
  "Filter: (l_shipdate > '1995-03-15'::date)"]]